### 0 Setup & configuration
Update INPUT_ROOT to your combined directory. OUTPUT_ROOT is a new working folder for processed outputs and caches.
Using Environment: 

In [7]:
INPUT_ROOT  = "/Users/stepanyan/Library/CloudStorage/OneDrive-UniversityCollegeLondon/Knowledge Exchange and Innovation - UCL - Wikimedia - Shared working space/NER/Newswire - Jupyter/Pi-OCR-Articles-Cleaned-Combined"
OUTPUT_ROOT = "/Users/stepanyan/Library/CloudStorage/OneDrive-UniversityCollegeLondon/Knowledge Exchange and Innovation - UCL - Wikimedia - Shared working space/NER/Newswire - Jupyter/pi-newswire-out"

import os, re, json, gzip, time, math, glob, random, textwrap, itertools
from pathlib import Path
from collections import defaultdict, Counter

import pandas as pd
from tqdm import tqdm
tqdm.pandas()

os.makedirs(OUTPUT_ROOT, exist_ok=True)
CACHE_DIR = Path(OUTPUT_ROOT) / "cache"
CACHE_DIR.mkdir(parents=True, exist_ok=True)

print("INPUT_ROOT :", INPUT_ROOT)
print("OUTPUT_ROOT:", OUTPUT_ROOT)


INPUT_ROOT : /Users/stepanyan/Library/CloudStorage/OneDrive-UniversityCollegeLondon/Knowledge Exchange and Innovation - UCL - Wikimedia - Shared working space/NER/Newswire - Jupyter/Pi-OCR-Articles-Cleaned-Combined
OUTPUT_ROOT: /Users/stepanyan/Library/CloudStorage/OneDrive-UniversityCollegeLondon/Knowledge Exchange and Innovation - UCL - Wikimedia - Shared working space/NER/Newswire - Jupyter/pi-newswire-out


### 1 Environment (installs)
This step is done further down the notebook, as the versions in the code below were incompatible. 

In [ ]:
# Light stack; British English (1940s–50s) will use general English models.
# %pip -q install spacy spacy-transformers transformers requests requests-cache geopy rapidfuzz

# # spaCy model (transformer is accurate; small corpus so speed is OK)
# !python -m spacy download en_core_web_trf



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Traceback (most recent call last):
  File "<frozen runpy>", line 189, in _run_module_as_main
  File "<frozen runpy>", line 148, in _get_module_details
  File "<frozen runpy>", line 112, in _get_module_details
  File "/opt/anaconda3/lib/python3.12/site-packages/spacy/__init__.py", line 13, in <module>
    from . import pipeline  # noqa: F401
    ^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.12/site-packages/spacy/pipeline/__init__.py", line 1, in <module>
    from .attributeruler import AttributeRuler
  File "/opt/anaconda3/lib/python3.12/site-packages/spacy/pipeline/attributeruler.py", line 8, in <module>
    from ..language import Language
  File "/opt/anaconda3/lib/python3.12/site-packages/spacy/language.py", line 46, in <module>
    from .pipe_analysis import analyze_pipes, print_pipe_analysis, va

### 2. Load combined JSONs → normalized rows


In [11]:
# --- Step 2 (patched): Load combined JSONs → normalized rows (handles {"articles":[...]} shape) ---
import json, re
from pathlib import Path
import pandas as pd

# Keys we’ll look for
TITLE_KEYS = ("title","headline","heading")
TEXT_KEYS  = ("content","body","text","article","main_text","raw_text")

def iter_records_from_obj(obj):
    """Yield article dicts from various container shapes."""
    if isinstance(obj, list):
        for it in obj:
            yield it
    elif isinstance(obj, dict):
        # Your combined files: top-level has {"articles":[{...}]}
        if "articles" in obj and isinstance(obj["articles"], list):
            for it in obj["articles"]:
                yield it
        else:
            yield obj
    else:
        # raw string line (rare)
        yield {"raw_text": str(obj)}

def read_any_json(path: Path):
    """Read a file that may be a single JSON, array JSON, or JSONL; return iterator of dicts."""
    txt = path.read_text(encoding="utf-8").strip()
    # Try full JSON first
    try:
        obj = json.loads(txt)
        for rec in iter_records_from_obj(obj):
            yield rec
        return
    except json.JSONDecodeError:
        pass
    # Fallback: JSONL
    for line in txt.splitlines():
        line = line.strip()
        if not line:
            continue
        try:
            obj = json.loads(line)
        except json.JSONDecodeError:
            obj = {"raw_text": line}
        for rec in iter_records_from_obj(obj):
            yield rec

def pick_title_text(d: dict):
    """Extract (title, body) from a single article record."""
    title = None
    for k in TITLE_KEYS:
        if k in d and isinstance(d[k], str) and d[k].strip():
            title = d[k].strip()
            break
    body = None
    for k in TEXT_KEYS:
        if k in d and isinstance(d[k], str) and d[k].strip():
            body = d[k].strip()
            break
    # Last resort: stack content-like fields if present
    if body is None and any(k in d for k in ("paragraphs","content_blocks")):
        body = json.dumps({k: d.get(k) for k in ("paragraphs","content_blocks")}, ensure_ascii=False)
    return title, body

def collect_articles(root: str) -> pd.DataFrame:
    rows = []
    for year_dir in sorted(Path(root).glob("[12][09][0-9][0-9]")):
        y = int(year_dir.name)
        for f in sorted(year_dir.glob("*-cleaned.json")):
            for i, it in enumerate(read_any_json(f)):
                title, body = pick_title_text(it)

                # Always capture id/page first (fixes UnboundLocalError)
                art_id = it.get("id", None)
                page   = it.get("page", None)

                # If still no body, keep it empty (flag later if desired)
                if not body:
                    body = ""

                rows.append({
                    "year": y,
                    "source_file": str(f),
                    "unit_index": i,
                    "title": title,
                    "text": body,
                    "id": art_id,
                    "page": page,
                    "orig": it,  # keep the original article dict for optional fields (author/date)
                    "orig_has_articles_wrapper": isinstance(it, dict) and ("articles" in it),
                })
    return pd.DataFrame(rows)

df = collect_articles(INPUT_ROOT)
print(len(df), "articles loaded")

# sanity check: NO escaped JSON in 'text'
print(df["text"].astype(str).str.startswith('{"articles"').sum(), "rows still look like a wrapper (should be 0)")
df.head(3)


3366 articles loaded
0 rows still look like a wrapper (should be 0)


,year,source_file,unit_index,title,text,id,page,orig,orig_has_articles_wrapper
0,1946,/Users/stepanyan/Library/CloudStorage/OneDrive...,0,Success To New College Newspaper 1 the Provost,In a College the size of ours communications a...,1,1,"{'id': 1, 'title': 'Success To New College New...",False
1,1946,/Users/stepanyan/Library/CloudStorage/OneDrive...,1,Next Month UCL Will Be 119 Years Old: Societie...,"PREPARATIONS for Foundation Week, which begins...",2,1,"{'id': 2, 'title': 'Next Month UCL Will Be 119...",False
2,1946,/Users/stepanyan/Library/CloudStorage/OneDrive...,2,ELECTION RESULTS,HERE are the official results of the Universit...,3,1,"{'id': 3, 'title': 'ELECTION RESULTS', 'author...",False


### 3. Light OCR cleanup (conservative)



In [12]:
def clean_ocr(t: str) -> str:
    # soft hyphen at EOL, hard hyphenation joins, normalize quotes & spaces
    t = t.replace("\u00ad\n", "")
    t = re.sub(r"(\w)-\n(\w)", r"\1\2", t)
    t = re.sub(r"[“”]", '"', t).replace("’", "'")
    t = re.sub(r"[ \t]+\n", "\n", t)
    t = re.sub(r"\n{3,}", "\n\n", t)
    return t.strip()

df["text_clean"] = df["text"].map(clean_ocr)
df["title_clean"] = df["title"].map(lambda s: clean_ocr(s) if isinstance(s,str) else None)
df["char_len"] = df["text_clean"].str.len()
df[["year","title_clean","char_len"]].head(5)


,year,title_clean,char_len
0,1946,Success To New College Newspaper 1 the Provost,1593
1,1946,Next Month UCL Will Be 119 Years Old: Societie...,2155
2,1946,ELECTION RESULTS,614
3,1946,Reconstruction Delayed By Housing Programme,1588
4,1946,EXAMINATIONS PREVENT MEDICS JOINING FUN,973


### 4. Geocoding


In [13]:
%pip install -U typing_extensions
%load_ext autoreload
%autoreload 2


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [14]:
# %pip install -U "pydantic>=2.7" "spacy>=3.7.4"
%pip install -U "typing_extensions>=4.12"
%pip install -U "spacy>=3.7.2" "pydantic>=2,<3" spacy-transformers transformers torch
# (Re)install an English model:
!python -m spacy download en_core_web_trf


import sys, importlib.metadata as im

print("Python:", sys.version)
print("Kernel executable:", sys.executable)
for pkg in ("spacy", "pydantic", "pydantic-core", "thinc"):
    try:
        print(f"{pkg}:", im.version(pkg))
    except im.PackageNotFoundError:
        print(f"{pkg}: not installed")



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 457.4/457.4 MB 28.6 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_trf')
Python: 3.12.4 | packaged by Anaconda, Inc. | (main, Jun 18 2024, 10:14:12) [Clang 14.0.6 ]
Kernel executable: /opt/anaconda3/bin/python
spacy: 3.8.7
pydantic: 2.11.7
pydantic-core: 2.33.2
thinc: 8.3.4


### 4A Extract all location mentions from the full text
RUN TIME - ~14 mins. 

In [15]:
# ---- Step 4A: Extract all location mentions across the full text ----
import re, json
from collections import Counter

# Ensure an English spaCy pipeline is loaded (we'll try transformer; fall back to small)
try:
    nlp
except NameError:
    import spacy
    try:
        nlp = spacy.load("en_core_web_trf")
    except Exception:
        nlp = spacy.load("en_core_web_sm")

# Simple UK/EN street patterns (extend as needed)
STREET_SUFFIXES = r"(Street|St\.|Road|Rd\.|Avenue|Ave\.|Lane|Ln\.|Close|Way|Square|Sq\.|Terrace|Terr\.|Place|Pl\.|Drive|Dr\.|Crescent|Cresc\.|Court|Ct\.|Row|Quay|Embankment|Parade)"
STREET_RE = re.compile(rf"\b([A-Z][a-zA-Z'-]+(?:\s+[A-Z][a-zA-Z'-]+)*)\s+{STREET_SUFFIXES}\b")

def extract_location_mentions(text: str):
    """
    Returns a list of candidate location mentions with offsets and a coarse 'kind'.
    Kinds: 'street', 'place' (GPE/LOC), 'facility' (FAC), 'country' (if resolved later).
    """
    out = []
    if not isinstance(text, str) or not text.strip():
        return out

    # 1) NER for GPE/LOC/FAC across entire text
    doc = nlp(text)
    for ent in doc.ents:
        if ent.label_ in ("GPE","LOC","FAC"):
            out.append({
                "text": ent.text.strip(".,;:()[]\"' "),
                "start": ent.start_char,
                "end": ent.end_char,
                "kind": "place" if ent.label_ in ("GPE","LOC") else "facility",
                "source": "ner"
            })

    # 2) Street patterns (regex), capture overlaps that spaCy might miss
    for m in STREET_RE.finditer(text):
        span = m.group(0).strip(".,;:()[]\"' ")
        out.append({
            "text": span,
            "start": m.start(),
            "end": m.end(),
            "kind": "street",
            "source": "regex"
        })

    # 3) Normalize & deduplicate near-duplicates (case-insensitive)
    # Keep first occurrence offsets
    seen = set()
    deduped = []
    for m in out:
        key = (m["text"].lower(), m["kind"])
        if key not in seen:
            seen.add(key)
            deduped.append(m)

    return deduped

# Apply to all rows
df["location_mentions"] = df["text_clean"].map(extract_location_mentions)

# Quick peek
df[["title_clean","location_mentions"]].head(10)


,title_clean,location_mentions
0,Success To New College Newspaper 1 the Provost,[]
1,Next Month UCL Will Be 119 Years Old: Societie...,"[{'text': 'Birmingham', 'start': 439, 'end': 4..."
2,ELECTION RESULTS,[]
3,Reconstruction Delayed By Housing Programme,"[{'text': 'the Main Building', 'start': 728, '..."
4,EXAMINATIONS PREVENT MEDICS JOINING FUN,[]
5,Photoelasticity: New Slant on Camera Technique,"[{'text': 'Athens', 'start': 128, 'end': 134, ..."
6,Six UCL Students Off to Switzerland,"[{'text': 'Carshalton', 'start': 83, 'end': 93..."
7,SPEAK UP!!,[]
8,WHY A NEWSPAPER?,"[{'text': 'the British Empire', 'start': 557, ..."
9,None,[]


### 4B. Geocode mentions → structured address (street/council/city/region/country)
RUN TIME ~35 mins. 



In [17]:
# ---- Step 4B: Geocode to structured fields with cache & rate limiter ----
import requests, requests_cache, json, time
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from pathlib import Path

# Persistent HTTP cache (2 weeks)
CACHE_DIR.mkdir(parents=True, exist_ok=True)
requests_cache.install_cache(str(CACHE_DIR / "http_cache"), expire_after=60*60*24*14)

# Local JSONL geocode cache
GEOCODE_CACHE = CACHE_DIR / "all_locations_geocode_cache.jsonl"
_geo_cache = {}
if GEOCODE_CACHE.exists():
    with open(GEOCODE_CACHE, "r", encoding="utf-8") as f:
        for line in f:
            rec = json.loads(line)
            _geo_cache[(rec["query"].lower(), rec.get("country_bias"))] = rec

def _cache_geocode_write(key, rec):
    _geo_cache[key] = rec
    with open(GEOCODE_CACHE, "a", encoding="utf-8") as f:
        f.write(json.dumps(rec, ensure_ascii=False) + "\n")

# Single shared Nominatim client with rate limiting
_geolocator = Nominatim(user_agent="pi-newswire-alllocs", timeout=15)
_geocode = RateLimiter(_geolocator.geocode, min_delay_seconds=1.0, swallow_exceptions=True)

def normalize_address(addr: dict) -> dict:
    """
    Map OSM address fields to our target schema:
      street, council, city, region, country, lat, lon
    """
    if not addr:
        return {}

    # Street candidates (choose in priority order)
    street = (addr.get("road") or addr.get("pedestrian") or addr.get("residential") or
              addr.get("footway") or addr.get("path") or addr.get("neighbourhood") or None)

    # Council (UK-style) from a range of possible keys
    council = (addr.get("borough") or addr.get("municipality") or addr.get("city_district") or
               addr.get("district") or addr.get("county") or addr.get("local_authority") or
               addr.get("state_district") or None)

    # City-ish (pick one)
    city = (addr.get("city") or addr.get("town") or addr.get("village") or
            addr.get("hamlet") or addr.get("suburb") or None)

    # Region / top-level within country (e.g., England/Scotland/Bavaria/Île-de-France)
    region = (addr.get("state") or addr.get("region") or addr.get("province") or None)

    country = addr.get("country")

    return {
        "street": street,
        "council": council,
        "city": city,
        "region": region,
        "country": country,
    }

def geocode_text(query: str, country_bias: str|None=None) -> dict|None:
    """
    Geocode a text mention via Nominatim, returning normalized address + lat/lon.
    country_bias: e.g., 'gb' to bias to the UK for ambiguous street names.
    """
    if not query or not query.strip():
        return None
    key = (query.lower(), country_bias)
    if key in _geo_cache:
        return _geo_cache[key]

    try:
        loc = _geocode(query, addressdetails=True, country_codes=country_bias)
    except Exception:
        loc = None

    if not loc:
        rec = {"query": query, "country_bias": country_bias, "ok": False}
        _cache_geocode_write(key, rec)
        return rec

    addr = normalize_address(getattr(loc, "raw", {}).get("address", {}))
    rec = {
        "query": query,
        "country_bias": country_bias,
        "ok": True,
        "lat": loc.latitude,
        "lon": loc.longitude,
        "address": addr,
        "display_name": loc.address,
        "source": "nominatim"
    }
    _cache_geocode_write(key, rec)
    return rec

def geocode_mentions_for_row(mentions, prefer_uk_streets=True):
    """
    For each mention, geocode and return a list of resolved locations.
    - For 'street' kinds, optionally bias to GB (prefer_uk_streets=True).
    - For 'place'/'facility', try without bias.
    Deduplicate by (lat,lon,address dict) to keep unique resolved points.
    """
    if not mentions:
        return []

    results = []
    seen_keys = set()

    for m in mentions:
        q = m["text"]
        kind = m.get("kind","place")
        bias = "gb" if (kind == "street" and prefer_uk_streets) else None

        rec = geocode_text(q, country_bias=bias)
        if not rec or not rec.get("ok"):
            continue

        addr = rec.get("address", {})
        lat, lon = rec.get("lat"), rec.get("lon")

        norm = {
            "mention": q,
            "kind": kind,
            "street": addr.get("street") or addr.get("road") or addr.get("pedestrian") or None,
            "council": addr.get("council") or addr.get("borough") or addr.get("municipality") or addr.get("city_district") or addr.get("district") or addr.get("county"),
            "city": addr.get("city") or addr.get("town") or addr.get("village") or addr.get("hamlet") or addr.get("suburb"),
            "region": addr.get("region") or addr.get("state") or addr.get("province"),
            "country": addr.get("country"),
            "lat": lat, "lon": lon,
            "display_name": rec.get("display_name"),
            "geo_source": rec.get("source"),
            "country_bias": rec.get("country_bias"),
        }

        # Build a stable dedup key
        dkey = (round(lat,6) if lat else None, round(lon,6) if lon else None,
                norm["street"], norm["council"], norm["city"], norm["region"], norm["country"])
        if dkey in seen_keys:
            continue
        seen_keys.add(dkey)
        results.append(norm)

    return results

# Apply to all rows
df["resolved_locations"] = df["location_mentions"].map(geocode_mentions_for_row)

# Inspect a few resolved examples
cols = ["title_clean","resolved_locations"]
df[cols].head(5)


,title_clean,resolved_locations
0,Success To New College Newspaper 1 the Provost,[]
1,Next Month UCL Will Be 119 Years Old: Societie...,"[{'mention': 'Birmingham', 'kind': 'place', 's..."
2,ELECTION RESULTS,[]
3,Reconstruction Delayed By Housing Programme,"[{'mention': 'the Main Building', 'kind': 'fac..."
4,EXAMINATIONS PREVENT MEDICS JOINING FUN,[]


In [18]:
# ---- Build locations_mentioned (one list per article) ----
from typing import List, Dict, Any

def _to_locations_mentioned(resolved_list: List[Dict[str, Any]] | None) -> List[Dict[str, Any]]:
    """
    Map our resolved_locations records to the target schema:
      {city, state, country, coordinates [lat, lon], location_notes}
    Deduplicate by (city, state, country, lat, lon).
    """
    out = []
    seen = set()
    if not isinstance(resolved_list, list):
        return out

    for r in resolved_list:
        city    = (r.get("city") or None)
        state   = (r.get("region") or None)   # treat 'region' as 'state'
        country = (r.get("country") or None)

        lat = r.get("lat"); lon = r.get("lon")
        coords = [float(lat), float(lon)] if (lat is not None and lon is not None) else None

        key = (city, state, country,
               round(float(lat), 6) if lat is not None else None,
               round(float(lon), 6) if lon is not None else None)
        if key in seen:
            continue
        seen.add(key)

        out.append({
            "city": city,
            "state": state,
            "country": country,
            "coordinates": coords,
            "location_notes": r.get("display_name") or ""
        })
    return out

df["locations_mentioned"] = df["resolved_locations"].map(_to_locations_mentioned)

# Quick sanity check
df[["locations_mentioned"]].head(3)


,locations_mentioned
0,[]
1,"[{'city': 'Birmingham', 'state': 'England', 'c..."
2,[]


In [19]:
# Not Sure what this is....
df.to_csv("locations-dump.csv", index=False, encoding="utf-8")

### 5. NER → BIO tags (Newswire-style fields)
RUN TIME ~14 mins.

In [20]:
import spacy
nlp = spacy.load("en_core_web_trf")

def ner_bio(text: str):
    doc = nlp(text)
    words = [t.text for t in doc]
    labels = ["O"] * len(words)
    for ent in doc.ents:
        print(f"Entity: {ent.text} ({ent.label_}) at {ent.start_char}-{ent.end_char}")
        labels[ent.start] = f"B-{ent.label_}"
        for i in range(ent.start+1, ent.end):
            labels[i] = f"I-{ent.label_}"
    ents = [(e.text, e.label_) for e in doc.ents]
    return words, labels, ents

# Store compactly; full arrays go to final record
df["_ner"] = df["text_clean"].progress_map(ner_bio)

  0%|          | 2/3366 [00:00<13:27,  4.16it/s]

Entity: College (ORG) at 5-12
Entity: University College (ORG) at 70-88
Entity: College (ORG) at 184-191
Entity: College (ORG) at 776-783
Entity: College (ORG) at 1021-1028
Entity: Pi (PERSON) at 1086-1088
Entity: the Editorial Committee (ORG) at 1119-1142
Entity: Court (ORG) at 1349-1354


  0%|          | 3/3366 [00:01<20:44,  2.70it/s]

Entity: Foundation Week (DATE) at 17-32
Entity: Monday, March 11th (DATE) at 47-65
Entity: the 18th (DATE) at 86-94
Entity: a fortnight (DATE) at 172-183
Entity: two (CARDINAL) at 226-229
Entity: pre-Foundation Week (DATE) at 230-249
Entity: the Foundation Ball (WORK_OF_ART) at 293-312
Entity: Foundation (ORG) at 358-368
Entity: this year (DATE) at 377-386
Entity: Barnes (PERSON) at 429-435
Entity: Birmingham (GPE) at 439-449
Entity: the Dramatic Society (ORG) at 482-502
Entity: five (CARDINAL) at 509-513
Entity: two (CARDINAL) at 547-550
Entity: between Wednesday, March 6th and March 8th (DATE) at 558-600
Entity: One (CARDINAL) at 602-605
Entity: Cain (WORK_OF_ART) at 629-633
Entity: Christopher Biddell (PERSON) at 648-667
Entity: The Lying Valet (WORK_OF_ART) at 674-689
Entity: Alan Barrow (PERSON) at 704-715
Entity: Saturday (DATE) at 838-846
Entity: the Q-Camps (ORG) at 905-916
Entity: Dramatic Society's (ORG) at 944-962
Entity: Foundation Week (DATE) at 976-991
Entity: Tuesday, Ma

  0%|          | 4/3366 [00:01<20:42,  2.71it/s]

...

Entity: Feb. 26 (DATE) at 0-7
Entity: UCL (ORG) at 9-12
Entity: 15 (CARDINAL) at 13-15
Entity: N. Staffs Eng. (ORG) at 17-31
Entity: 0 (CARDINAL) at 38-39
Entity: The First XV (ORG) at 42-54
Entity: Northampton Engineering College (ORG) at 91-122
Entity: Tufnell Park (FAC) at 140-152
Entity: Jones (PERSON) at 154-159
Entity: UCL (ORG) at 183-186
Entity: only two minutes (TIME) at 193-209
Entity: Brown (PERSON) at 215-220
Entity: Carter (PERSON) at 256-262
Entity: Turner (PERSON) at 264-270
Entity: Jones (PERSON) at 275-280
Entity: Brown (PERSON) at 317-322
Entity: UCL (ORG) at 433-436
Entity: 12 (CARDINAL) at 437-439
Entity: Northampton Eng. (ORG) at 441-457
Entity: 3 (CARDINAL) at 467-468
Entity: seconds (ORG) at 475-482
Entity: Hurner (PERSON) at 497-503
Entity: Smith (PERSON) at 556-561
Entity: 2 (CARDINAL) at 563-564
Entity: Davies (PERSON) at 570-576
Entity: Brown (PERSON) at 600-605
Entity: Jennifer Copeman (PERSON) at 17-33
Entity: ULC (ORG) at 56-59
Entity: Monday, March 1st (D

100%|██████████| 3366/3366 [15:25<00:00,  4.12it/s]

Entity: the second year running (DATE) at 4-27
Entity: the University League Championship (EVENT) at 64-98
Entity: League (ORG) at 134-140
Entity: Goldsmith's College (ORG) at 209-228
Entity: City of London College (ORG) at 234-256
Entity: 6–1 (CARDINAL) at 257-260
Entity: Battersea College (ORG) at 262-279
Entity: 5–2 (CARDINAL) at 280-283
Entity: LSE (ORG) at 285-288
Entity: 7–0 (CARDINAL) at 289-292
Entity: RCS (ORG) at 294-297
Entity: 5–2 (CARDINAL) at 298-301
Entity: UEC (ORG) at 303-306
Entity: 6–1 (CARDINAL) at 307-310
Entity: P. Sadler (PERSON) at 321-330
Entity: S. Allen (PERSON) at 379-387
Entity: M. Rachel (PERSON) at 392-401


100%|██████████| 3366/3366 [15:25<00:00,  3.64it/s]

Entity: THURSDAY, 4th MARCH
5.30 (DATE) at 0-24
Entity: Math. Soc (ORG) at 25-34
Entity: 8.15 Dispute & Physics Soc (ORG) at 57-83
Entity: FRIDAY, 5th MARCH
 (DATE) at 102-120
Entity: SATURDAY, 6th MARCH
 (DATE) at 139-159
Entity: ULC Women's Fives (ORG) at 164-181
Entity: Queen Mary College (ORG) at 184-202
Entity: 5.30 (DATE) at 203-207
Entity: Imperial College (ORG) at 219-235
Entity: SUNDAY, 7th MARCH (DATE) at 237-254
Entity: Arts Soc (ORG) at 261-269
Entity: MONDAY, 8th MARCH (DATE) at 293-310
Entity: Electoral Hall (FAC) at 340-354
Entity: Emu Ballroom (FAC) at 376-388
Entity: TUESDAY, 9th MARCH
6.00 (DATE) at 390-413
Entity: Senior Common Room (FAC) at 442-460
Entity: WEDNESDAY, 10th MARCH
6.30 (DATE) at 462-488
Entity: THURSDAY, 11th MARCH
 (DATE) at 520-541
Entity: University Players (ORG) at 546-564
Entity: Main Theatre (FAC) at 567-579


In [21]:
df["_ner"].to_csv("ner-dump.csv", index=False, encoding="utf-8")

### 6. People disambiguation to Wikidata (cached, local-first)
RUN TIME 1.3 secs.

In [22]:
# ---- Step 6: People linking (cache keyed by mention; cache *record* uses canonical 'name') ----
import json, re, requests
from rapidfuzz import fuzz

PEOPLE_CACHE = CACHE_DIR / "people_link_cache.jsonl"

def _norm_key(s: str) -> str:
    return re.sub(r"\s+", " ", (s or "")).strip()

# In-memory cache, looked up by the *mention* text (normalized)
_people_store: dict[str, dict | None] = {}

# Fresh start is fine if you deleted the cache file; loader is tolerant anyway
if PEOPLE_CACHE.exists():
    with open(PEOPLE_CACHE, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            try:
                rec = json.loads(line)
            except Exception:
                continue
            if not isinstance(rec, dict):
                continue
            # Prefer original mention to be the key if available; fall back safely
            key = rec.get("query") or rec.get("name") or rec.get("person_name")
            if key:
                _people_store[_norm_key(key)] = rec

def _cache_person_write(mention: str, resolved: dict | None):
    """
    Write one JSONL record per mention:
      - If resolved: include canonical 'name' (Wikidata label) and the original 'query' (mention).
      - If unresolved: write a minimal record with 'name': None.
    """
    PEOPLE_CACHE.parent.mkdir(parents=True, exist_ok=True)
    key = _norm_key(mention)

    if resolved is None:
        rec = {"query": mention, "name": None}
        _people_store[key] = rec
    else:
        # Ensure canonical 'name' present; keep mention in 'query'
        canon = resolved.get("person_name") or resolved.get("name") or mention
        rec = {
            "query": mention,
            "name": canon,                 # canonical label
            **resolved
        }
        _people_store[key] = rec

    with open(PEOPLE_CACHE, "a", encoding="utf-8") as f:
        f.write(json.dumps(rec, ensure_ascii=False) + "\n")

def wb_search(name: str, limit=5):
    url = "https://www.wikidata.org/w/api.php"
    params = {
        "action": "wbsearchentities",
        "language": "en",
        "uselang": "en",
        "type": "item",
        "search": name,
        "limit": limit,
        "format": "json",
    }
    r = requests.get(url, params=params, headers={"User-Agent": "pi-newswire-pipeline"})
    if not r.ok:
        return []
    return r.json().get("search", [])

def is_human_qid(qid: str) -> bool:
    # SPARQL: instance of human (Q5)
    sparql = f"""
    ASK {{ wd:{qid} wdt:P31 wd:Q5 . }}
    """
    url = "https://query.wikidata.org/sparql"
    r = requests.get(url, params={"format": "json", "query": sparql}, headers={"User-Agent": "pi-newswire-pipeline"})
    if not r.ok:
        return False
    try:
        return bool(r.json().get("boolean", False))
    except Exception:
        return False

def link_person(mention: str, context: str | None = None):
    """
    Resolve a PERSON mention to Wikidata.
    Cache lookup is by *mention* (normalized).
    Cache record stores the canonical 'name' (Wikidata label) and keeps 'query'=mention.
    Returns a dict like {"wikidata_id": "Qxx", "person_name": "..."} or None.
    """
    key = _norm_key(mention)
    if key in _people_store:
        return _people_store[key] if _people_store[key].get("name") else None

    cands = wb_search(mention, limit=7)
    best = None
    best_score = -1

    for c in cands:
        qid = c.get("id")
        if not qid:
            continue
        label = c.get("label") or ""
        desc  = c.get("description") or ""
        score = fuzz.token_set_ratio(mention, label)
        if score > best_score:
            best = {"wikidata_id": qid, "person_name": label, "desc": desc}
            best_score = score

    # Validate human & write cache
    if best and best.get("wikidata_id") and is_human_qid(best["wikidata_id"]):
        out = {
            "wikidata_id": best["wikidata_id"],
            "person_name": best["person_name"]
            # Optionally enrich later: gender, occupations, etc.
        }
        _cache_person_write(mention, out)
        return out

    _cache_person_write(mention, None)
    return None

def people_from_ner(ents):
    # Two-token+ names only (avoid bare surnames); adjust if you want
    return sorted({text for (text, label) in ents if label == "PERSON" and len(text.split()) >= 2})

# Quick smoke test on first row:
sample_words, sample_labels, sample_ents = df["_ner"].iloc[0]
people = people_from_ner(sample_ents)
people[:10]


[]

### 7. Topic tagging (Newswire heads)
RUN TIME ~15 mins?

In [19]:
# Previous version of transformers did not work. The suggested solution was to update it. 
%pip install -U "transformers>=4.41" "huggingface_hub>=0.23" "safetensors>=0.4.3" "torch>=2.1,<3"
# CHECK again BEFORE USING

  Using cached transformers-4.54.1-py3-none-any.whl.metadata (41 kB)
Using cached transformers-4.54.1-py3-none-any.whl (11.2 MB)
  Attempting uninstall: transformers
    Found existing installation: transformers 4.49.0
    Uninstalling transformers-4.49.0:
      Successfully uninstalled transformers-4.49.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spacy-transformers 1.3.9 requires transformers<4.50.0,>=3.4.0, but you have transformers 4.54.1 which is incompatible.

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


# ############
# in BASH
##############

Installing models locally

###############

# 1) Install the CLI (in the same environment as your Jupyter kernel)
pip install -U "huggingface_hub[cli]" hf_transfer

# 2) Faster transfer (optional)
#export HF_HUB_ENABLE_HF_TRANSFER=1

# 3) Choose a local model directory
 MODEL_HOME="$HOME/hf-models/newswire"
 mkdir -p "$MODEL_HOME"

# 4) Download each repo to its own folder
 for m in \
   dell-research-harvard/topic-antitrust \
   dell-research-harvard/topic-civil_rights \
   dell-research-harvard/topic-crime \
   dell-research-harvard/topic-govt_regulation \
   dell-research-harvard/topic-labor_movement \
   dell-research-harvard/topic-politics \
   dell-research-harvard/topic-protests \
   dell-research-harvard/topic-sport \
   dell-research-harvard/topic-fire \
   dell-research-harvard/topic-weather \
   dell-research-harvard/topic-obits
 do
   name="${m##*/}"
   huggingface-cli download "$m" --local-dir "$MODEL_HOME/$name" --local-dir-use-symlinks False
 done
 


In [30]:
# ---- Step 7: Topic tagging with ALL Newswire heads (local-first, safe, cached) ----
from transformers import pipeline
from pathlib import Path
import json, time

# 0) Ensure text column is strings
df["text_clean"] = df["text_clean"].fillna("").astype(str)

# 1) Point to where you stored the models (EDIT THIS to your path if different)
# MODEL_HOME = Path.home() / "hf-models" / "newswire"   # e.g., Path("/Users/stepanyan/hf-models/newswire")
MODEL_HOME = Path("/Users/stepanyan/Library/CloudStorage/OneDrive-UniversityCollegeLondon/Knowledge Exchange and Innovation - UCL - Wikimedia - Shared working space/NER/Newswire - Jupyter/Newswire-Topic-Models/newswire")

# 2) All Newswire topic heads
TOPIC_MODELS = {
    "antitrust":        "dell-research-harvard/topic-antitrust",
    "civil_rights":     "dell-research-harvard/topic-civil_rights",
    "crime":            "dell-research-harvard/topic-crime",
    "govt_regulation":  "dell-research-harvard/topic-govt_regulation",
    "labor_movement":   "dell-research-harvard/topic-labor_movement",
    "politics":         "dell-research-harvard/topic-politics",
    "protests":         "dell-research-harvard/topic-protests",
    "sport":            "dell-research-harvard/topic-sport",
    "fire":             "dell-research-harvard/topic-fire",
    "weather":          "dell-research-harvard/topic-weather",
    "obits":            "dell-research-harvard/topic-obits",
}

# Helper: prefer local folder if present
def resolve_model_id(mid: str) -> str:
    local = MODEL_HOME / mid.split("/", 1)[1]
    return str(local) if local.exists() else mid

# If you want to force strictly-offline (error if model missing locally), set:
STRICT_OFFLINE = False  # True -> raise if local folder missing

DEVICE    = -1      # -1 = CPU; set 0 for CUDA-GPU, or None to auto-pick (MPS on Apple Silicon)
BATCH_SIZE= 8
TRUNC     = 4000
THRESH    = 0.5

_topic_pipes = {}
def get_pipe(model_id: str):
    model_ref = resolve_model_id(model_id)
    if STRICT_OFFLINE and not Path(model_ref).exists():
        raise FileNotFoundError(f"Local model not found: {model_ref}")
    if model_ref in _topic_pipes:
        return _topic_pipes[model_ref]
    try:
        _topic_pipes[model_ref] = pipeline("text-classification", model=model_ref, device=DEVICE)
    except Exception as e:
        print(f"[warn] Could not load {model_ref}: {type(e).__name__}: {e}")
        _topic_pipes[model_ref] = None
    return _topic_pipes[model_ref]

def _to_binary_label(hf_output, threshold=THRESH):
    if not hf_output:
        return 0
    rec = hf_output[0] if isinstance(hf_output, list) else hf_output
    label = str(rec.get("label","")).upper()
    score = float(rec.get("score", 0.0))
    is_pos = ("1" in label) or ("POS" in label) or ("YES" in label)
    return 1 if (is_pos and score >= threshold) else 0

def score_one_model(model_id: str, texts: list[str]) -> list[int]:
    clf = get_pipe(model_id)
    if clf is None:
        return [0]*len(texts)
    # retry once on transient errors
    try:
        preds = clf([ (t or "")[:TRUNC] for t in texts ], batch_size=BATCH_SIZE, truncation=True)
    except Exception:
        time.sleep(1.0)
        preds = clf([ (t or "")[:TRUNC] for t in texts ], batch_size=BATCH_SIZE, truncation=True)
    return [_to_binary_label(p) for p in preds]

# Cache predictions so re-runs are quick
CACHE_PATH = Path(OUTPUT_ROOT) / "cache" / "topic_preds.jsonl"
CACHE_PATH.parent.mkdir(parents=True, exist_ok=True)
_cache = {}
if CACHE_PATH.exists():
    with open(CACHE_PATH, "r", encoding="utf-8") as f:
        for line in f:
            j = json.loads(line)
            _cache[(j["source_file"], int(j["unit_index"]))] = j["topics"]

def _write_cache_row(key, topics_dict):
    with open(CACHE_PATH, "a", encoding="utf-8") as f:
        f.write(json.dumps({"source_file": key[0], "unit_index": key[1], "topics": topics_dict}) + "\n")

# Mini-batch over rows to reduce per-model calls
B = 32
pending_keys, pending_texts = [], []
topics_all = []

def flush_pending():
    global pending_keys, pending_texts, topics_all
    if not pending_texts:
        return
    batch = {t: score_one_model(mid, pending_texts) for t, mid in TOPIC_MODELS.items()}
    for i, key in enumerate(pending_keys):
        row_topics = {t: int(batch[t][i]) for t in TOPIC_MODELS}
        topics_all.append(row_topics)
        _write_cache_row(key, row_topics)
    pending_keys.clear(); pending_texts.clear()

for _, row in df.iterrows():
    key = (row["source_file"], int(row["unit_index"]))
    if key in _cache:
        topics_all.append(_cache[key])
        continue
    pending_keys.append(key); pending_texts.append(row["text_clean"])
    if len(pending_texts) >= B:
        flush_pending()
flush_pending()

df["topics"] = topics_all
for t in TOPIC_MODELS:
    df[t] = df["topics"].map(lambda d: int(d.get(t, 0)))

# Quick tally so you can see it worked
display(df[[*TOPIC_MODELS.keys()]].sum())


antitrust           77
civil_rights         0
crime               79
govt_regulation     36
labor_movement     305
politics           137
protests            24
sport              439
fire                13
weather              1
obits                9
dtype: int64

### 8.A Build Newswire-like records & export year-bucketed JSONL


In [28]:
def to_newswire_like(row):
    words, labels, ents = row["_ner"]
    ppl   = people_from_ner(ents)
    linked = [link_person(p, row["text_clean"]) for p in ppl]
    linked = [x for x in linked if x]

    return {
        "year": int(row["year"]),
        "dates": [],
        "article": row["text_clean"],
        "byline": "",
        "newspaper_metadata": [],
        "source_file": row.get("source_file"),  # full path or filename from Step 2
        "art_id": row.get("id"),                # map DF 'id' -> JSONL 'art_id'
        "page": row.get("page"),                # map DF 'page' -> JSONL 'page'

        # topics (present if you ran Step 7)
        "antitrust":        int(row.get("antitrust", 0)),
        "civil_rights":     int(row.get("civil_rights", 0)),
        "crime":            int(row.get("crime", 0)),
        "govt_regulation":  int(row.get("govt_regulation", 0)),
        "labor_movement":   int(row.get("labor_movement", 0)),
        "politics":         int(row.get("politics", 0)),
        "protests":         int(row.get("protests", 0)),
        "sport":            int(row.get("sport", 0)),
        "fire":             int(row.get("fire", 0)),
        "weather":          int(row.get("weather", 0)),
        "obits":            int(row.get("obits", 0)),

        # NER (BIO)
        "ner_words":  words,
        "ner_labels": labels,

        # New: all resolved locations
        "locations_mentioned": row.get("locations_mentioned", []),

        # Optional: keep for audit (not required in the final JSON)
        # "resolved_locations": row.get("resolved_locations", []),

        # People
        "people_mentioned": linked,
    }


out_dir = Path(OUTPUT_ROOT) / "pi_newswire_like"
out_dir.mkdir(parents=True, exist_ok=True)

def write_year_buckets(frame: pd.DataFrame):
    for y, g in frame.groupby("year"):
        outp = out_dir / f"{y}_pi_newswire_like.jsonl"
        with open(outp, "w", encoding="utf-8") as f:
            for _, row in tqdm(g.iterrows(), total=len(g), desc=f"Writing {y}"):
                f.write(json.dumps(to_newswire_like(row), ensure_ascii=False) + "\n")
        print("Wrote", outp)

write_year_buckets(df)


Writing 1946: 100%|██████████| 359/359 [00:00<00:00, 4935.59it/s]


Wrote /Users/stepanyan/Library/CloudStorage/OneDrive-UniversityCollegeLondon/Knowledge Exchange and Innovation - UCL - Wikimedia - Shared working space/NER/Newswire - Jupyter/pi-newswire-out/pi_newswire_like/1946_pi_newswire_like.jsonl


Writing 1947: 100%|██████████| 293/293 [00:00<00:00, 2626.39it/s]


Wrote /Users/stepanyan/Library/CloudStorage/OneDrive-UniversityCollegeLondon/Knowledge Exchange and Innovation - UCL - Wikimedia - Shared working space/NER/Newswire - Jupyter/pi-newswire-out/pi_newswire_like/1947_pi_newswire_like.jsonl


Writing 1948: 100%|██████████| 568/568 [00:00<00:00, 4913.33it/s]


...

Wrote /Users/stepanyan/Library/CloudStorage/OneDrive-UniversityCollegeLondon/Knowledge Exchange and Innovation - UCL - Wikimedia - Shared working space/NER/Newswire - Jupyter/pi-newswire-out/pi_newswire_like/1952_pi_newswire_like.jsonl


Writing 1953: 100%|██████████| 372/372 [00:00<00:00, 5119.40it/s]


Wrote /Users/stepanyan/Library/CloudStorage/OneDrive-UniversityCollegeLondon/Knowledge Exchange and Innovation - UCL - Wikimedia - Shared working space/NER/Newswire - Jupyter/pi-newswire-out/pi_newswire_like/1953_pi_newswire_like.jsonl


Writing 1954: 100%|██████████| 196/196 [00:00<00:00, 2419.12it/s]

Wrote /Users/stepanyan/Library/CloudStorage/OneDrive-UniversityCollegeLondon/Knowledge Exchange and Innovation - UCL - Wikimedia - Shared working space/NER/Newswire - Jupyter/pi-newswire-out/pi_newswire_like/1954_pi_newswire_like.jsonl


### 8.B Build Newswire-like records & export Issue-bucketed JSONL

In [29]:
# --- Step 8B: Export issue-based JSONL (preserve year folders + filename) ---
from pathlib import Path

out_dir_issue = Path(OUTPUT_ROOT) / "pi_newswire_by_issue"
out_dir_issue.mkdir(parents=True, exist_ok=True)

def write_issue_buckets(frame: pd.DataFrame):
    # Keep stable ordering within each issue
    sort_cols = [c for c in ["year", "source_file", "unit_index"] if c in frame.columns]
    gframe = frame.sort_values(sort_cols, kind="stable")

    # Group by issue (source_file) within each year
    for (y, src), g in gframe.groupby(["year", "source_file"], sort=False):
        year_dir = out_dir_issue / str(int(y))
        year_dir.mkdir(parents=True, exist_ok=True)

        # Keep original filename but change extension to .jsonl
        fname = Path(src).with_suffix(".jsonl").name
        outp = year_dir / fname

        with open(outp, "w", encoding="utf-8") as f:
            for _, row in tqdm(g.iterrows(), total=len(g), desc=f"Writing {fname}"):
                f.write(json.dumps(to_newswire_like(row), ensure_ascii=False) + "\n")

        print("Wrote", outp)

write_issue_buckets(df)


Writing pi_1_1946-cleaned.jsonl: 100%|██████████| 25/25 [00:00<00:00, 3286.04it/s]


Wrote /Users/stepanyan/Library/CloudStorage/OneDrive-UniversityCollegeLondon/Knowledge Exchange and Innovation - UCL - Wikimedia - Shared working space/NER/Newswire - Jupyter/pi-newswire-out/pi_newswire_by_issue/1946/pi_1_1946-cleaned.jsonl


Writing pi_2_1946-cleaned.jsonl: 100%|██████████| 45/45 [00:00<00:00, 3307.12it/s]


Wrote /Users/stepanyan/Library/CloudStorage/OneDrive-UniversityCollegeLondon/Knowledge Exchange and Innovation - UCL - Wikimedia - Shared working space/NER/Newswire - Jupyter/pi-newswire-out/pi_newswire_by_issue/1946/pi_2_1946-cleaned.jsonl


Writing pi_3_1946-cleaned.jsonl: 100%|██████████| 27/27 [00:00<00:00, 2240.46it/s]


...

Wrote /Users/stepanyan/Library/CloudStorage/OneDrive-UniversityCollegeLondon/Knowledge Exchange and Innovation - UCL - Wikimedia - Shared working space/NER/Newswire - Jupyter/pi-newswire-out/pi_newswire_by_issue/1954/pi_vol_11_6_1954-cleaned.jsonl


Writing pi_vol_11_7_1954-cleaned.jsonl: 100%|██████████| 61/61 [00:00<00:00, 2184.96it/s]


Wrote /Users/stepanyan/Library/CloudStorage/OneDrive-UniversityCollegeLondon/Knowledge Exchange and Innovation - UCL - Wikimedia - Shared working space/NER/Newswire - Jupyter/pi-newswire-out/pi_newswire_by_issue/1954/pi_vol_11_7_1954-cleaned.jsonl


Writing pi_vol_11_8_1954-cleaned.jsonl: 100%|██████████| 30/30 [00:00<00:00, 2087.06it/s]


Wrote /Users/stepanyan/Library/CloudStorage/OneDrive-UniversityCollegeLondon/Knowledge Exchange and Innovation - UCL - Wikimedia - Shared working space/NER/Newswire - Jupyter/pi-newswire-out/pi_newswire_by_issue/1954/pi_vol_11_8_1954-cleaned.jsonl


### 9. Per-issue summary (Topics, NERs, Geolocations)


In [26]:
import re
from pathlib import Path
from collections import Counter
import pandas as pd

# -------- Helper: issue id from filename --------
ISSUE_RX = re.compile(r"^pi_(.+?)_(19|20)\d\d$")  # captures between 'pi_' and '_YYYY'
def issue_id_from_path(p: str) -> str:
    stem = Path(p).stem  # e.g., 'pi_vol_2_3_1948-cleaned' -> 'pi_vol_2_3_1948-cleaned'
    stem = stem.replace("-cleaned","")
    m = ISSUE_RX.match(stem)
    if m:
        return m.group(1)  # e.g., '1', 'vol_2_3', '19_first_edition', 'rag'
    # fallback: remove prefix 'pi_' and year suffix
    s = stem
    if s.startswith("pi_"): s = s[3:]
    s = re.sub(r"_(19|20)\d\d$", "", s)
    return s

df["issue_id"] = df["source_file"].map(issue_id_from_path)

# -------- Topics per issue --------
topic_cols_all = [
    "antitrust","civil_rights","crime","govt_regulation","labor_movement",
    "politics","protests","sport","fire","weather","obits"
]
topic_cols = [c for c in topic_cols_all if c in df.columns]

def topics_for_group(g: pd.DataFrame):
    present = [t for t in topic_cols if g[t].sum() > 0]
    return present, len(present)

# -------- NERs per issue (top 25 by frequency) --------
ALLOWED_NER = {"PERSON","ORG","GPE","LOC","FAC","EVENT","WORK_OF_ART","LAW","NORP"}
def ners_for_group(g: pd.DataFrame, topk=25):
    cnt = Counter()
    for item in g["_ner"]:
        if not isinstance(item, (list, tuple)) or len(item) < 3:
            continue
        _, _, ents = item
        for text, label in ents:
            if label in ALLOWED_NER:
                norm = text.strip().strip('.,;:"\'()[]')
                if norm:
                    cnt[norm] += 1
    n_unique = len(cnt)
    listed = [f"{k} ({v})" for k,v in cnt.most_common(topk)]
    return listed, n_unique

# -------- Geolocations per issue (top 20) --------
def geos_for_group(g: pd.DataFrame, topk=20):
    cnt = Counter()
    for L in g.get("resolved_locations", []):
        if not isinstance(L, list):
            continue
        for r in L:
            parts = [r.get("street"), r.get("council"), r.get("city"), r.get("region"), r.get("country")]
            name = ", ".join([p for p in parts if p])
            if not name:
                name = r.get("display_name") or ""
            name = name.strip().strip(",")
            if name:
                cnt[name] += 1
    n_unique = len(cnt)
    listed = [f"{k} ({v})" for k,v in cnt.most_common(topk)]
    return listed, n_unique

# -------- People Mentioned per issue (top 20) --------
def people_for_group(g: pd.DataFrame, topk=20):
    cnt = Counter()

    # Prefer the explicit column if it exists; otherwise derive from NER
    if "people_mentioned" in g.columns:
        series = g["people_mentioned"]
    else:
        # Fallback: derive from NER spans (expects people_from_ner and _ner available)
        def _from_ner(item):
            if not isinstance(item, (list, tuple)) or len(item) < 3:
                return []
            _, _, ents = item
            try:
                names = people_from_ner(ents)  # returns list of strings
            except NameError:
                names = [text for (text, label) in ents if label == "PERSON" and len(text.split()) >= 2]
            return [{"person_name": n} for n in names]
        series = g["_ner"].map(_from_ner) if "_ner" in g.columns else pd.Series([[]]*len(g), index=g.index)

    for L in series:
        if not isinstance(L, list):
            continue
        for r in L:
            # r may be a dict from linker, or a bare string if upstream changed
            if isinstance(r, dict):
                name = r.get("person_name") or r.get("name") or ""
            else:
                name = str(r) if r is not None else ""
            name = re.sub(r"\s+", " ", name).strip().strip('.,;:"\'()[]')
            if name:
                cnt[name] += 1

    n_unique = len(cnt)
    listed = [f"{k} ({v})" for k, v in cnt.most_common(topk)]
    return listed, n_unique
    

# -------- Build summary table --------
rows = []
for (year, issue), g in df.groupby(["year","issue_id"], sort=True):
    topics_list, topics_n = topics_for_group(g)
    ners_list, ners_n     = ners_for_group(g)
    geos_list, geos_n     = geos_for_group(g)
    people_list, people_n = people_for_group(g)
    rows.append({
        "Year": year,
        "Issue": issue,
        "List of Topics": ", ".join(topics_list),
        "Number of Topics": topics_n,
        "List of NERs (top 25)": "; ".join(ners_list),
        "Number of NERs (unique)": ners_n,
        "List of Geolocations (top 20)": "; ".join(geos_list),
        "Number of Geolocations (unique)": geos_n,
        "List of People Mentioned (top 20)": "; ".join(people_list),
        "Number of People Mentioned (unique)": people_n,
        "Articles in Issue": len(g),
    })

summary = pd.DataFrame(rows).sort_values(["Year","Issue"]).reset_index(drop=True)
summary
summary.to_csv("pi_newswire_output_summary.csv", index=False, encoding="utf-8")  

# Convert JSONL output to CSV for Wikidata upload
## Extract People Mentioned
This section iterates through the files in pi-newswire-out->pi_newswire_like and creates a single 


In [33]:
# --- Step 9: Extract people from JSONL into a CSV (basename + issue) ---
import json
import pandas as pd
from pathlib import Path

people_rows = []

# Path to Step 8 JSONL exports
jsonl_dir = Path(OUTPUT_ROOT) / "pi_newswire_like"
CSV_DIR = "/Users/stepanyan/Library/CloudStorage/OneDrive-UniversityCollegeLondon/Knowledge Exchange and Innovation - UCL - Wikimedia - Shared working space/3-Wikidata-Publishing"
out_csv = Path(CSV_DIR) / "people_mentioned.csv"

def make_issue(name: str | None) -> str | None:
    """From a filename (no dirs), strip '-cleaned.jsonl' (preferred), else '-cleaned.json',
    else just remove .jsonl/.json. Return None if name is falsy."""
    if not name:
        return None
    s = name
    for suf in ("-cleaned.jsonl", "-cleaned.json", ".jsonl", ".json"):
        if s.endswith(suf):
            s = s[: -len(suf)]
            break
    return s

for f in sorted(jsonl_dir.glob("*.jsonl")):
    with open(f, "r", encoding="utf-8") as fh:
        for line in fh:
            if not line.strip():
                continue
            rec = json.loads(line)

            year        = rec.get("year")
            sf_full     = rec.get("source_file")
            sf_name     = Path(sf_full).name if sf_full else None  # [1] basename only
            issue_name  = make_issue(sf_name)                      # [2] derived issue

            art_id      = rec.get("art_id")
            page        = rec.get("page")

            # Expecting a list of dicts in "people_mentioned"
            for p in rec.get("people_mentioned", []):
                # Defensive parsing: handle dicts or plain strings
                if isinstance(p, dict):
                    name        = p.get("name")
                    wikidata_id = p.get("wikidata_id")
                    person_name = p.get("person_name", name)
                else:
                    # fallback if it's just a string
                    name        = str(p)
                    wikidata_id = None
                    person_name = str(p)

                people_rows.append({
                    "year": year,
                    "source_file": sf_name,  # basename only
                    "issue": issue_name,     # new column
                    "art_id": art_id,
                    "page": page,
                    "name": name,
                    "wikidata_id": wikidata_id,
                    "person_name": person_name,
                })

# Convert to DataFrame
people_df = pd.DataFrame(people_rows)

# Save to CSV
people_df.to_csv(out_csv, index=False, encoding="utf-8")
print(f"Wrote {len(people_df)} rows to {out_csv}")

people_df.head(10)


Wrote 1629 rows to /Users/stepanyan/Library/CloudStorage/OneDrive-UniversityCollegeLondon/Knowledge Exchange and Innovation - UCL - Wikimedia - Shared working space/3-Wikidata-Publishing/people_mentioned.csv


,year,source_file,issue,art_id,page,name,wikidata_id,person_name
0,1946,pi_1_1946-cleaned.json,pi_1_1946,2,1,Alan Barrow,Q4706138,Alan Barrow
1,1946,pi_1_1946-cleaned.json,pi_1_1946,3,1,Hannah E Steinberg,Q87147930,Hannah E Steinberg
2,1946,pi_1_1946-cleaned.json,pi_1_1946,13,2,Betty Grable,Q233852,Betty Grable
3,1946,pi_1_1946-cleaned.json,pi_1_1946,13,2,Bob Hope,Q94081,Bob Hope
4,1946,pi_1_1946-cleaned.json,pi_1_1946,13,2,Hedy Lamarr,Q49034,Hedy Lamarr
5,1946,pi_1_1946-cleaned.json,pi_1_1946,13,2,J. Morris,Q135174365,J. Morris
6,1946,pi_1_1946-cleaned.json,pi_1_1946,14,2,Frank Dutton,Q5486350,Frank Dutton
7,1946,pi_1_1946-cleaned.json,pi_1_1946,17,2,Frank Dutton,Q5486350,Frank Dutton
8,1946,pi_1_1946-cleaned.json,pi_1_1946,20,3,Jack Chapman,Q100769030,Jack Chapman
9,1946,pi_1_1946-cleaned.json,pi_1_1946,20,3,Norman Burton,Q4351837,Norman Burton


# Convert JSONL output to CSV for Wikidata upload
### Extracting Locations mentioned 

In [34]:
# --- Step 10: Extract locations from JSONL into a CSV (with components) ---
import json
import pandas as pd
from pathlib import Path

# ---- Inputs/outputs
jsonl_dir = Path(OUTPUT_ROOT) / "pi_newswire_like"
CSV_DIR   = "/Users/stepanyan/Library/CloudStorage/OneDrive-UniversityCollegeLondon/Knowledge Exchange and Innovation - UCL - Wikimedia - Shared working space/3-Wikidata-Publishing"
out_csv   = Path(CSV_DIR) / "locations_mentioned.csv"

# ---- Helpers
def make_issue(name: str | None) -> str | None:
    """From a filename (no dirs), strip '-cleaned.jsonl' (preferred), else '-cleaned.json',
    else just remove .jsonl/.json. Return None if name is falsy."""
    if not name:
        return None
    s = Path(name).name
    for suf in ("-cleaned.jsonl", "-cleaned.json", ".jsonl", ".json"):
        if s.endswith(suf):
            return s[: -len(suf)]
    return Path(s).stem

def to_float(x):
    try:
        return float(x)
    except Exception:
        return None

def parse_location(loc) -> dict:
    """
    Normalize a location entry (dict or str) to:
    city, state, country, longitude, latitude, location_notes
    Handles common shapes (Nominatim-like dicts, custom dicts, or plain strings).
    """
    # Defaults
    out = {
        "city": None,
        "state": None,
        "country": None,
        "longitude": None,
        "latitude": None,
        "location_notes": None,
    }

    # If it's a plain string, store it as a note
    if not isinstance(loc, dict):
        out["location_notes"] = str(loc)
        return out

    d = loc

    # 1) Direct fields (your own structured output)
    out["city"]    = d.get("city")    or d.get("town") or d.get("village") or d.get("municipality")
    out["state"]   = d.get("state")   or d.get("region") or d.get("province")
    out["country"] = d.get("country")

    # 2) Coordinates (various key spellings)
    lon = d.get("longitude", d.get("lon", d.get("lng")))
    lat = d.get("latitude",  d.get("lat"))
    # GeoJSON-ish geometry
    if lon is None or lat is None:
        geom = d.get("geometry") or d.get("geo") or d.get("point")
        if isinstance(geom, dict):
            # e.g. {"coordinates":[lon,lat]} or {"lon":x,"lat":y}
            if "coordinates" in geom and isinstance(geom["coordinates"], (list, tuple)) and len(geom["coordinates"]) >= 2:
                lon = geom["coordinates"][0]
                lat = geom["coordinates"][1]
            else:
                lon = lon or geom.get("lon") or geom.get("lng")
                lat = lat or geom.get("lat")

    out["longitude"] = to_float(lon)
    out["latitude"]  = to_float(lat)

    # 3) Nominatim-style address
    addr = d.get("address")
    if isinstance(addr, dict):
        out["city"]    = out["city"]    or addr.get("city") or addr.get("town") or addr.get("village")
        out["state"]   = out["state"]   or addr.get("state") or addr.get("region") or addr.get("province")
        out["country"] = out["country"] or addr.get("country")

    # 4) Notes / display name / source / confidence
    notes_parts = []
    for key in ("display_name", "note", "notes", "source", "confidence", "type"):
        v = d.get(key)
        if v is not None and str(v).strip():
            notes_parts.append(f"{key}={v}")
    out["location_notes"] = "; ".join(notes_parts) if notes_parts else d.get("location_notes")

    return out

# ---- Main extraction
rows = []

for f in sorted(jsonl_dir.glob("*.jsonl")):
    with open(f, "r", encoding="utf-8") as fh:
        for line in fh:
            if not line.strip():
                continue
            rec = json.loads(line)

            year        = rec.get("year")
            sf_full     = rec.get("source_file")
            sf_name     = Path(sf_full).name if sf_full else None
            issue_name  = make_issue(sf_name)

            art_id      = rec.get("art_id")
            page        = rec.get("page")
            article     = rec.get("article")  # full text

            # locations may be saved under a few different keys; try them in order
            loc_list = (
                rec.get("locations_mentioned")
                or rec.get("_locations_structured")
                or rec.get("resolved_locations")
                or rec.get("locations")
                or []
            )

            if not isinstance(loc_list, list):
                loc_list = [loc_list]

            for loc in loc_list:
                norm = parse_location(loc)
                rows.append({
                    "year": year,
                    "source_file": sf_name,    # basename only
                    "issue": issue_name,       # derived issue name
                    "art_id": art_id,
                    "article": article,
                    "page": page,
                    "city": norm["city"],
                    "state": norm["state"],
                    "country": norm["country"],
                    "longitude": norm["longitude"],
                    "latitude": norm["latitude"],
                    "location_notes": norm["location_notes"],
                })

# ---- DataFrame + save
loc_df = pd.DataFrame(rows)
out_csv.parent.mkdir(parents=True, exist_ok=True)
loc_df.to_csv(out_csv, index=False, encoding="utf-8")
print(f"Wrote {len(loc_df)} rows to {out_csv}")

loc_df.head(10)


Wrote 3841 rows to /Users/stepanyan/Library/CloudStorage/OneDrive-UniversityCollegeLondon/Knowledge Exchange and Innovation - UCL - Wikimedia - Shared working space/3-Wikidata-Publishing/locations_mentioned.csv


,year,source_file,issue,art_id,article,page,city,state,country,longitude,latitude,location_notes
0,1946,pi_1_1946-cleaned.json,pi_1_1946,2,"PREPARATIONS for Foundation Week, which begins...",1,Birmingham,England,United Kingdom,None,None,"Birmingham, West Midlands, England, United Kin..."
1,1946,pi_1_1946-cleaned.json,pi_1_1946,2,"PREPARATIONS for Foundation Week, which begins...",1,Patna,Bihar,India,None,None,"friend's house, Patna, Patna Rural, Patna, Bih..."
2,1946,pi_1_1946-cleaned.json,pi_1_1946,2,"PREPARATIONS for Foundation Week, which begins...",1,City of London,England,United Kingdom,None,None,"St. Paul's, Panyer Alley, St Paul's, City of L..."
3,1946,pi_1_1946-cleaned.json,pi_1_1946,4,PLANS for the rebuilding of University College...,1,London,England,United Kingdom,None,None,"The Main Building, Brook Green, London Borough..."
4,1946,pi_1_1946-cleaned.json,pi_1_1946,4,PLANS for the rebuilding of University College...,1,Dome,Greater Accra Region,Ghana,None,None,"Dome, Ga East Municipal District, Greater Accr..."
5,1946,pi_1_1946-cleaned.json,pi_1_1946,4,PLANS for the rebuilding of University College...,1,London,England,United Kingdom,None,None,"Gordon Street, Saint Pancras, London Borough o..."
6,1946,pi_1_1946-cleaned.json,pi_1_1946,4,PLANS for the rebuilding of University College...,1,South Kesteven,England,United Kingdom,None,None,"Main Block, South Road, Elsea Park, Bourne, Au..."
7,1946,pi_1_1946-cleaned.json,pi_1_1946,4,PLANS for the rebuilding of University College...,1,Hobart,Tasmania,Australia,None,None,"State Library of Tasmania, Bathurst Street, Ho..."
8,1946,pi_1_1946-cleaned.json,pi_1_1946,4,PLANS for the rebuilding of University College...,1,Jersey Marine,Cymru / Wales,United Kingdom,None,None,"Gower Square, Coedffranc, Jersey Marine, Neath..."
9,1946,pi_1_1946-cleaned.json,pi_1_1946,6,"Mr. Costa Mylonas, a post-graduate engineering...",1,Athens,Georgia,United States,None,None,"Athens, Clarke County, Georgia, United States"
